https://medium.com/jbennetcodes/how-to-rewrite-your-sql-queries-in-pandas-and-more-149d341fc53e

In [ ]:
import pandas as pd

airports = pd.read_csv('data/airports.csv')
airport_freq = pd.read_csv('data/airport-frequencies.csv')
runways = pd.read_csv('data/runways.csv')

# SELECT, WHERE, DISTINCT, LIMIT

In [ ]:
# select * from airports
airports

In [ ]:
#select * from airports limit 3
airports.head(3)

In [ ]:
#select id from airports where ident = 'KLAX'
airports[airports.ident == 'KLAX'].id

In [ ]:
#select distinct type from airport
airports.type.unique()

In [ ]:
### SELECT with multiple conditions
# select * from airports where iso_region = 'US-CA' and type = 'seaplane_base'

airports[(airports.iso_region == 'US-CA') & (airports.type == 'seaplane_base')]


In [ ]:
# select ident, name, municipality from airports where iso_region = 'US-CA' and type = 'large_airport'

airports[(airports.iso_region == 'US-CA') & (airports.type == 'large_airport')][['ident', 'name', 'municipality']]

#  ORDER BY

In [ ]:
# select * from airport_freq where airport_ident = 'KLAX' order by type

airport_freq[airport_freq.airport_ident=='KLAX'].sort_values('type')

In [ ]:
# select * from airport_freq where airport_ident = 'KLAX' order by type desc

airport_freq[airport_freq.airport_ident=='KLAX'].sort_values('type', ascending=False)

# IN… NOT IN

In [ ]:
# select * from airports where type in ('heliport', 'balloonport')
airports[airports.type.isin(['heliport', 'balloonport'])]

In [ ]:
# select * from airports where type not in ('heliport', 'balloonport')

airports[~airports.type.isin(['heliport', 'balloonport'])]

# GROUP BY, COUNT, ORDER BY

Grouping is straightforward: use the .groupby() operator. There’s a subtle difference between semantics of a COUNT in SQL and Pandas. In Pandas, .count() will return the number of non-null/NaN values. To get the same result as the SQL COUNT, use .size().

SQL count:  COUNT(*) = 4;                        -- count all rows, even null/duplicates

            COUNT(Field1) = COUNT(Field2) = 3;   -- count only rows without null values on that field
            
pandas.count(): Count non-NA cells for each column or row

In [ ]:
# select iso_country, type, count(*) from airports group by iso_country, type order by iso_country, type

airports.groupby(['iso_country', 'type']).size()


In [ ]:
airports.groupby(['iso_country', 'type']).count()

In [ ]:
# select iso_country, type, count(*) from airports group by iso_country, type order by iso_country, count(*) desc
airports.groupby(['iso_country', 'type']).size().to_frame('size').reset_index().sort_values(['iso_country', 'size'], ascending=[True, False])

# HAVING
In SQL, you can additionally filter grouped data using a HAVING condition. In Pandas, you can use .filter() and provide a Python function (or a lambda) that will return True if the group should be included into the result.

select type, count(*) from airports where iso_country = 'US' group by type having count(*) > 1000 order by count(*) desc

In [ ]:
airports[airports.iso_country == 'US'].groupby('type').filter(lambda g: len(g) > 1000).groupby('type').size().sort_values(ascending=False)

# Top N records

In [ ]:
by_country=airports.groupby('iso_country').size().to_frame('airport_count').reset_index().sort_values('iso_country')

In [ ]:
by_country.head(3)

In the next example, we order things by airport_count and only select the top 10 countries with the largest count. Second example is the more complicated case, in which we want “the next 10 after the top 10”:

In [ ]:
# select iso_country from by_country order by size desc limit 10

by_country.nlargest(10, columns='airport_count')


In [ ]:
# select iso_country from by_country order by size desc limit 10 offset 10

by_country.nlargest(20, columns='airport_count').tail(10)


# Aggregate functions (MIN, MAX, MEAN)

In [ ]:
runways.head(3)

In [ ]:
#select max(length_ft), min(length_ft), avg(length_ft), median(length_ft) from runways

df=runways.agg({'length_ft': ['min', 'max', 'mean', 'median']})
df.T

# JOIN

select airport_ident, type, description, frequency_mhz 
from airport_freq 
join airports 
on airport_freq.airport_ref = airports.id 
where airports.ident = 'KLAX'

In [ ]:
airport_freq.merge(airports[airports.ident == 'KLAX'][['id']], left_on='airport_ref', right_on='id', how='inner')[['airport_ref','airport_ident', 'type', 'description', 'frequency_mhz']]

In [ ]:
airport_freq.merge(airports[airports['ident'] == 'KLAX'][['id']], left_on='airport_ref', right_on='id', how='inner')


In [ ]:
airport_freq.merge(airports[airports.ident == 'KLAX'], left_on='airport_ref', right_on='id', how='inner')

In [ ]:
airports[airports.ident == 'KLAX'][['id']]

# UNION ALL and UNION
Use pd.concat() to UNION ALL two dataframes


To deduplicate things (equivalent of UNION), you’d also have to add .drop_duplicates()


select name, municipality 
from airports 
where ident = 'KLAX' 
union all 

select name, municipality 
from airports 
where ident = 'KLGB'

In [ ]:
pd.concat([airports[airports.ident == 'KLAX'][['name', 'municipality']], airports[airports.ident == 'KLGB'][['name', 'municipality']]])

# INSERT

create table heroes (id integer, name text);
insert into heroes values (1, 'Harry Potter');
insert into heroes values (2, 'Ron Weasley');
insert into heroes values (3, 'Hermione Granger');

In [ ]:
df1 = pd.DataFrame({'id': [1, 2], 'name': ['Harry Potter', 'Ron Weasley']})
df1

In [ ]:
df2 = pd.DataFrame({'id': [3], 'name': ['Hermione Granger']})
df2

In [ ]:
pd.concat([df1, df2]).reset_index(drop=True)

In [ ]:
pd.concat([df1, df2]).reset_index(drop=False)

In [ ]:
# Update

update airports set home_link = 'http://www.lawa.org/welcomelax.aspx' where ident == 'KLAX'

In [ ]:
airports.loc[airports['ident'] == 'KLAX', 'home_link'] = 'http://www.lawa.org/welcomelax.aspx'

In [ ]:
airports.loc[airports['ident'] == 'KLAX']

# DELETE
The easiest (and the most readable) way to “delete” things from a Pandas dataframe is to subset the dataframe to rows you want to keep. Alternatively, you can get the indices of rows to delete, and .drop() rows using those indices

delete from lax_freq where type = 'MISC'

In [ ]:
lax_freq = lax_freq[lax_freq.type != 'MISC']

In [ ]:
lax_freq.drop(lax_freq[lax_freq.type == 'MISC'].index)

# Immutability

I need to mention one important thing — immutability. By default, most operators applied to a Pandas dataframe return a new object. Some operators accept a parameter inplace=True, so you can work with the original dataframe instead. For example, here is how you would reset an index in-place:

In [ ]:
df.reset_index(drop=True, inplace=True)

However, the .loc operator in the UPDATE example above simply locates indices of records to updates, and the values are changed in-place. Also, if you updated all values in a column:

In [ ]:
df['url'] = 'http://google.com'

or added a new calculated column, these things would happen in-place.

In [ ]:
df['total_cost'] = df['price'] * df['quantity']

# And More

In [ ]:
df.to_csv(...)  # csv file
df.to_hdf(...)  # HDF5 file
df.to_pickle(...)  # serialized object
df.to_sql(...)  # to SQL database
df.to_excel(...)  # to Excel sheet
df.to_json(...)  # to JSON string
df.to_html(...)  # render as HTML table
df.to_feather(...)  # binary feather-format
df.to_latex(...)  # tabular environment table
df.to_stata(...)  # Stata binary data files
df.to_msgpack(...)	# msgpack (serialize) object
df.to_gbq(...)  # to a Google BigQuery table.
df.to_string(...)  # console-friendly tabular output.
df.to_clipboard(...) # clipboard that can be pasted into Excel

In [ ]:
top_10.plot(
    x='iso_country', 
    y='airport_count',
    kind='barh',
    figsize=(10, 7),
    title='Top 10 countries with most airports')